In [12]:
%load_ext autoreload
%autoreload 2
# coding=utf-8#%%

import defs as d
import os
import pandas as pd
# Use the LightGBM Tuner
# https://medium.com/optuna/lightgbm-tuner-new-optuna-integration-for-hyperparameter-optimization-8b7095e99258
# import optuna.integration.lightgbm as lgb_tuner
import lightgbm as lgb
import loss_functions as lf
import base_datasource as bd
import time_series_cross_validation as cv
import json
import optuna
import numpy as np
import parallel as pl
from joblib import delayed
d

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'defs' from '/home/jovyan/dev/projects/vinx/defs.py'>

In [13]:
%%time
print('start')
data = pd.read_pickle(d.TMP+'/merged_data.pkl')
data.shape



start
CPU times: user 75.8 ms, sys: 194 ms, total: 270 ms
Wall time: 535 ms


(210958, 50)

In [14]:
%%time
print('start')
def objective(trial):
  params = {
    'objective': 'gamma',
    'metric': 'fair',
    'verbosity': -1,
    'boosting_type': 'gbdt',

    # Bagging will randomly select part of data without resampling.
    # With time-series nature of our problem, bagging should be disabled.
    'bagging_fraction': 1,
    'bagging_freq': 0,

    # This parameter should not be zero (meaning no limit).
    # Need to find the best interval.
    'max_depth': 0,

    # Shrinkage rate
    'learning_rate': trial.suggest_uniform('learning_rate', 0.05, 0.2),

    # Take boosting_round large and early-stop if the metric doesn't improve.
    # 'num_boost_round': trial.suggest_int('num_boost_round', 100, 200),
    'num_boost_round': 400,
    'early_stopping': 10,

    # Each value of `store_cd` should have its own branch.
    # If you accept this claim, there should at least be 185 leaves(region).
    # If you split 2 region for each feature, then there will be 2^num_features regions(leaves).
    'num_leaves': trial.suggest_int('num_leaves', 400, 600),

    # Regularization parameters.
    'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
    'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),

    # Fraction of features that lgbm uses for each training.
    # Lower value means faster training.
    # seed value is the default = 2
    'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 0.8),
    'feature_fraction_seed': 2,

    # This parameter is for ensuring LLN should not be randomly tuned.
    'min_data_in_leaf': trial.suggest_categorical('min_data_in_leaf', [20, 50, 100]),
  }

  estimator = lgb.LGBMRegressor()
  metric = lf.mape
  scores = cv.cross_val_score(
    estimator
    ,params
    ,metric
    ,data
    ,lgbm_tuner=False
    ,random_seed=1
    ,n_jobs=4
    ,verbosity=50
  )

  # Store results in a external storage
  scores_df = pd.DataFrame(scores)
  mask = (scores_df['score'] >= 0)
  scores_df = scores_df[mask].copy()
  scores_df['trial_number'] = 10
  scores_df['trial_params'] = json.dumps(trial.params)
  scores_df['mean_score'] = scores_df['score'].mean()
  if trial.number == 0:
    cmd = "echo 'trial_number, split_date, score, mean_socre, params' > "+d.TMP+'/trial.csv'
    os.system(cmd)
    scores_df.to_csv(d.TMP+'/trial.csv', index=False)
  else:
    old_scores_df = pd.read_csv(d.TMP+'/trial.csv')
    merged_scores = pd.concat([old_scores_df, scores_df], axis=0, ignore_index=True)
    merged_scores.to_csv(d.TMP+'/trial.csv', index=False)

  return scores_df['score'].mean()



start
CPU times: user 785 µs, sys: 1.03 ms, total: 1.82 ms
Wall time: 1.76 ms


In [15]:
%%time

# 3h 9min 51s
# 5h 9min 21s
print('start')
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1, n_jobs=1)
print('end')


start
valid_y.shape:  5328
Non-Zero cnt:  5328
Score: 0.04627608714525071
valid_y.shape:  5425
Non-Zero cnt:  5425
Score: 0.0383842774009598
valid_y.shape:  5688
Non-Zero cnt:  5688
Score: 0.0916744896401746
valid_y.shape:  5735
Non-Zero cnt:  5735
Score: 0.04199747153639968
valid_y.shape:  5735
Non-Zero cnt:  5735
Score: 0.041567232976027595
valid_y.shape:  5766
Non-Zero cnt:  5766
Score: 0.06363607604359114
valid_y.shape:  5766
Non-Zero cnt:  5766
Score: 0.05423668216311856
valid_y.shape:  5762
Non-Zero cnt:  5762
Score: 0.061614965632212554
valid_y.shape:  5790
Non-Zero cnt:  5790
Score: 0.08358664632697484
valid_y.shape:  5795
Non-Zero cnt:  5795
Score: 0.09622933533641168
valid_y.shape:  5797
Non-Zero cnt:  5797
Score: 0.041299780668369916
valid_y.shape:  5904
Non-Zero cnt:  5809
end
CPU times: user 17min 8s, sys: 25.6 s, total: 17min 34s
Wall time: 2min 15s


/opt/conda/envs/azureml/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning:

Found `num_boost_round` in params. Will use it instead of argument

/opt/conda/envs/azureml/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning:

Found `early_stopping` in params. Will use it instead of argument

/opt/conda/envs/azureml/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning:

Found `num_boost_round` in params. Will use it instead of argument

/opt/conda/envs/azureml/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning:

Found `early_stopping` in params. Will use it instead of argument

/opt/conda/envs/azureml/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning:

Found `num_boost_round` in params. Will use it instead of argument

/opt/conda/envs/azureml/lib/python3.6/site-packages/lightgbm/engine.py:153: UserWarning:

Found `early_stopping` in params. Will use it instead of argument

/opt/conda/envs/azureml/lib/python3.6/site-packages/lig

In [21]:
%%time
print('start')
# pd.DataFrame(study.best_params)
json.dumps(study.best_params, indent=2)

start
CPU times: user 3.58 ms, sys: 6 µs, total: 3.59 ms
Wall time: 2.94 ms


'{\n  "learning_rate": 0.1543194350518498,\n  "num_leaves": 520,\n  "lambda_l1": 0.2971416053315697,\n  "lambda_l2": 0.020970380921065086,\n  "feature_fraction": 0.5972032021074605,\n  "min_data_in_leaf": 100\n}'

In [18]:
%%time
print('start')
df = study.trials_dataframe()
df.to_csv(d.TMP+'/trial_100_sampling_122_horizon_30_lgbm.csv', index=False)

start
CPU times: user 30.7 ms, sys: 0 ns, total: 30.7 ms
Wall time: 42.8 ms
